# Preparing the datasets for FFNN

In [ ]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
from textblob import Word
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,classification_report
from sklearn.metrics import recall_score,precision_score,precision_recall_fscore_support

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
#reading csv file
def read_corpus(file):
  return pd.read_csv(file)
  
train_data = '/content/gdrive/MyDrive/NLP_Project/train_data_processed.csv'
test_data = '/content/gdrive/MyDrive/NLP_Project/test_data_processed.csv'
test_true = '/content/gdrive/MyDrive/NLP_Project/Test_Actual_Final.csv'

In [ ]:
#Dataset containing the meme ground truth 

true_df = read_corpus(test_true)
#Extracting the first digit (1, 0 , -1) from Labels 

true_df['Sentiment'] = true_df['Labels'].str.split('_').str[0]
true_df['Sentiment'] = true_df['Sentiment'].astype(int)
true_df.head()

,Unnamed: 0,Image_name,Labels,Sentiment
0,0,chuck_chuck_norris_meme_10.jpg,1_1100_1100,1
1,1,dr_evil_NDBB96K.png,1_0100_0200,1
2,2,misog_2109e457d636565e2e06dce39874c5231e1.jpg,1_1110_1120,1
3,3,obama_2691536739_469698809820026_263513986_n.jpg,0_1111_1121,0
4,4,kim_threat-kim-jong-un-allegedly-working-on-mu...,0_0000_0000,0


In [ ]:
#Dataset containing the Train data
train_df = read_corpus(train_data)

train_df.head()

,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment,processed
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive,"['look', 'friend', 'lightyear', 'sohalikut', '..."
1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive,"['best', 'yearchallenge', 'complete', 'less', ..."
2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive,"['sam', 'thorne', 'strippin', 'follow', 'follo..."
3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive,"['year', 'challenge', 'sweet', 'dee', 'edition']"
4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral,"['year', 'challenge', 'filter', 'hilarious', '..."


In [ ]:
train_df['overall_sentiment'].value_counts()

positive         3057
neutral          2157
very_positive    1001
negative          469
very_negative     146
Name: overall_sentiment, dtype: int64

In [ ]:
#converting categorical to numerical 

train_df['overall_sentiment'].replace({'very_negative': -1, 'negative': -1, 'neutral': 0, 'positive': 1, 'very_positive': 1}, inplace=True)
train_df['overall_sentiment'].value_counts()

 1    4058
 0    2157
-1     615
Name: overall_sentiment, dtype: int64

In [ ]:
print(train_df.humour.value_counts())
print(train_df.sarcasm.value_counts())
print(train_df.offensive.value_counts())
print(train_df.motivational.value_counts())

funny         2394
very_funny    2176
not_funny     1618
hilarious      642
Name: humour, dtype: int64
general            3430
not_sarcastic      1516
twisted_meaning    1499
very_twisted        385
Name: sarcasm, dtype: int64
not_offensive        2657
slight               2536
very_offensive       1424
hateful_offensive     213
Name: offensive, dtype: int64
not_motivational    4421
motivational        2409
Name: motivational, dtype: int64


In [ ]:
train_df.humour[train_df['humour']!='not_funny']= 1
train_df.humour[train_df['humour']=='not_funny']= 0
train_df.sarcasm[train_df['sarcasm']!='not_sarcastic']= 1
train_df.sarcasm[train_df['sarcasm']=='not_sarcastic']= 0
train_df.offensive[train_df['offensive']!='not_offensive']= 1
train_df.offensive[train_df['offensive']=='not_offensive']= 0
train_df.motivational[train_df['motivational']!='not_motivational']= 1
train_df.motivational[train_df['motivational']=='not_motivational']= 0

In [ ]:
train_df.head()

,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment,processed
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,1,1,0,0,1,"['look', 'friend', 'lightyear', 'sohalikut', '..."
1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,0,1,0,1,1,"['best', 'yearchallenge', 'complete', 'less', ..."
2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,1,0,0,0,1,"['sam', 'thorne', 'strippin', 'follow', 'follo..."
3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,1,1,1,1,1,"['year', 'challenge', 'sweet', 'dee', 'edition']"
4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,1,1,1,0,0,"['year', 'challenge', 'filter', 'hilarious', '..."


In [ ]:
print(train_df.humour.value_counts())
print(train_df.sarcasm.value_counts())
print(train_df.offensive.value_counts())
print(train_df.motivational.value_counts())

1    5212
0    1618
Name: humour, dtype: int64
1    5314
0    1516
Name: sarcasm, dtype: int64
1    4173
0    2657
Name: offensive, dtype: int64
0    4421
1    2409
Name: motivational, dtype: int64


In [ ]:
#Dataset containing the processed text of test data

test_df = read_corpus(test_data)
test_df.head()

,Unnamed: 0,Image_name,Image_URL,OCR_extracted_text,corrected_text,processed
0,0,chuck_chuck_norris_meme_10.jpg,https://gtmemes.com/wp-content/uploads/2019/03...,Some magicians can walk on water Chuck Norris...,Some magicians can walk on water Chuck Norris...,"['magician', 'walk', 'water', 'chuck', 'norris..."
1,1,dr_evil_NDBB96K.png,https://i.imgur.com/NDBB96K.png,ONE MILLION DOLLARS made on imgur,ONE MILLION DOLLARS made on imgur,"['one', 'million', 'dollar', 'make', 'imgur']"
2,2,misog_2109e457d636565e2e06dce39874c5231e1.jpg,https://media0ch-a.akamaihd.net/83/96/9e457d63...,Me: Mom can my friend sleep over? Mom: That's ...,Me: Mom can my friend sleep over? Mom: That's ...,"['mom', 'friend', 'sleep', 'mom', 'fine', 'boy..."
3,3,obama_2691536739_469698809820026_263513986_n.jpg,http://politicalmemes.com/wp-content/uploads/2...,THIS GUY INHERITED A MESS. DID HE WHINE ABOUT ...,THIS GUY INHERITED A MESS. DID HE WHINE ABOUT ...,"['guy', 'inherit', 'mess', 'whine', 'foxed', '..."
4,4,kim_threat-kim-jong-un-allegedly-working-on-mu...,https://pics.me.me/threat-kim-jong-un-allegedl...,THREAT: Kim Jong Un allegedly working on multi...,THREAT: Kim Jong Un allegedly working on multi...,"['threat', 'kim', 'jong', 'un', 'allegedly', '..."


# Task1 using FFNN 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from gensim.utils import simple_preprocess
import gensim
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
from gensim.parsing.porter import PorterStemmer
porter_stemmer = PorterStemmer()      

In [ ]:

# Use cuda if present
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device available for running: ")
print(device)

Device available for running: 
cpu


In [ ]:
train_df_sub = train_df
train_df_sub.head()

,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment,processed
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,1,1,0,0,1,"['look', 'friend', 'lightyear', 'sohalikut', '..."
1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,0,1,0,1,1,"['best', 'yearchallenge', 'complete', 'less', ..."
2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,1,0,0,0,1,"['sam', 'thorne', 'strippin', 'follow', 'follo..."
3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,1,1,1,1,1,"['year', 'challenge', 'sweet', 'dee', 'edition']"
4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,1,1,1,0,0,"['year', 'challenge', 'filter', 'hilarious', '..."


In [ ]:
#preprocessing train data
def preprocess(train_df_sub, label):
  # Tokenize the text column to get the new column 'tokenized_text'
  train_df_sub['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in train_df_sub[label]] 

  #remove stop words
  train_df_sub['stop_tokens'] = [[remove_stopwords(word) for word in tokens] for tokens in train_df_sub['tokenized_text'] ]

  #remove punctuations
  train_df_sub['rem_punct_tokens'] = [[word.lower() for word in tokens if word.isalpha()] for tokens in train_df_sub['stop_tokens'] ]

  # Get the stemmed_tokens
  train_df_sub['pre_tokens'] = [[porter_stemmer.stem(word) for word in tokens] for tokens in train_df_sub['rem_punct_tokens'] ]
  
  return train_df_sub
  #train_df_sub['pre_tokens'].head(10)

train_df_sub = preprocess(train_df_sub, 'text_ocr')

train_df_sub.to_csv('train_data_processed.csv')

train_df_sub.head()

,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment,processed,tokenized_text,stop_tokens,rem_punct_tokens,pre_tokens
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,1,1,0,0,1,"['look', 'friend', 'lightyear', 'sohalikut', '...","[look, there, my, friend, lightyear, now, all,...","[look, , , friend, lightyear, , , sohalikut, t...","[look, friend, lightyear, sohalikut, trend, pl...","[look, friend, lightyear, sohalikut, trend, pl..."
1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,0,1,0,1,1,"['best', 'yearchallenge', 'complete', 'less', ...","[the, best, of, yearchallenge, completed, in, ...","[, best, , yearchallenge, completed, , , , yea...","[best, yearchallenge, completed, years, kudus,...","[best, yearchalleng, complet, year, kudu, nare..."
2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,1,0,0,0,1,"['sam', 'thorne', 'strippin', 'follow', 'follo...","[sam, thorne, strippin, follow, follow, saw, e...","[sam, thorne, strippin, follow, follow, saw, ,...","[sam, thorne, strippin, follow, follow, saw, p...","[sam, thorn, strippin, follow, follow, saw, po..."
3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,1,1,1,1,1,"['year', 'challenge', 'sweet', 'dee', 'edition']","[year, challenge, sweet, dee, edition]","[year, challenge, sweet, dee, edition]","[year, challenge, sweet, dee, edition]","[year, challeng, sweet, dee, edit]"
4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,1,1,1,0,0,"['year', 'challenge', 'filter', 'hilarious', '...","[year, challenge, with, no, filter, hilarious,...","[year, challenge, , , filter, hilarious, year,...","[year, challenge, filter, hilarious, year, cha...","[year, challeng, filter, hilari, year, challen..."


In [ ]:
#The FFNN model

class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FeedforwardNeuralNetModel, self).__init__()
        
        # Linear function 1: 
        self.l1 = nn.Linear(input_size, hidden_size) 
        # Non-linearity 1
        self.relu1 = nn.ReLU()

        self.l2 = nn.Linear(hidden_size, hidden_size)
        # Non-linearity 2
        self.relu2 = nn.ReLU()

        # Linear function 2 
        self.l3 = nn.Linear(hidden_size, num_classes)  

    def forward(self, x):
        out = self.l1(x)
        out = self.relu1(out)
        out = self.l2(out)
        out = self.relu2(out)
        out = self.l3(out)
        return out

In [ ]:
#Creating a dictionary of words 

from gensim import corpora
# Function to return the dictionary either with padding word or without padding
def make_dict(train_df_sub, padding=True):
    if padding:
        print("Dictionary with padded token added")
        review_dict = corpora.Dictionary([['pad']])
        review_dict.add_documents(train_df_sub['pre_tokens'])
    else:
        print("Dictionary without padding")
        review_dict = corpora.Dictionary(train_df_sub['pre_tokens'])
    return review_dict

# Make the dictionary without padding for the basic models
review_dict = make_dict(train_df_sub, padding=False)

Dictionary without padding


In [ ]:
#creating bag of words 

VOCAB_SIZE = len(review_dict)

# Function to make bow vector to be used as input to network
def make_bow_vector(review_dict, sentence):
    vec = torch.zeros(VOCAB_SIZE, dtype=torch.float64, device=device)
    for word in sentence:
        vec[review_dict.token2id[word]] += 1
    return vec.view(1, -1).float()

def make_bow_vector1(review_dict, sentence):
    vec = torch.zeros(VOCAB_SIZE, dtype=torch.float64, device=device)
    for word in sentence:
      if word in review_dict.keys():
        vec[review_dict.token2id[word]] += 1
    return vec.view(1, -1).float()

In [ ]:
# Function to get the output tensor
def make_target(label):
    if label == -1:
        return torch.tensor([2], dtype=torch.long, device=device)
    elif label == 0:
        return torch.tensor([0], dtype=torch.long, device=device)
    else:
        return torch.tensor([1], dtype=torch.long, device=device)

In [ ]:
#Training the FFNN model

VOCAB_SIZE = len(review_dict)

input_size = VOCAB_SIZE
hidden_size = 500
num_classes = 3
num_epochs = 100

ff_nn_bow_model = FeedforwardNeuralNetModel(input_size, hidden_size, num_classes)
ff_nn_bow_model.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(ff_nn_bow_model.parameters(), lr=0.01)
#optimizer = torch.optim.Adam(ff_nn_bow_model.parameters(), lr=0.001)

In [ ]:

X_train = train_df_sub[['pre_tokens']]
sentiment = train_df_sub['overall_sentiment']

# Start training
for epoch in range(num_epochs):
    if (epoch+1) % 25 == 0:
        print("Epoch completed: " + str(epoch+1))
    train_loss = 0
    for index, row in X_train.iterrows():
        # Clearing the accumulated gradients
        optimizer.zero_grad()

        # Make the bag of words vector for stemmed tokens 
        bow_vec = make_bow_vector(review_dict, row['pre_tokens'])
       
        # Forward pass to get output
        probs = ff_nn_bow_model(bow_vec)
        

        # Get the target label
        lab = train_df_sub['overall_sentiment'][index]
        target = make_target(lab)

        # Calculate Loss: softmax --> cross entropy loss
        loss = loss_function(probs, target)
        # Accumulating the loss over time
        train_loss += loss.item()

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

In [ ]:
test_df_sub = test_df
#test_df_sub

In [ ]:
#Preprocessing of test data
test_df_sub = preprocess(test_df_sub, 'OCR_extracted_text')
test_df_sub.to_csv('test_data_processed.csv')

In [ ]:
test_df_sub.head()

In [ ]:
from sklearn.metrics import classification_report
bow_ff_nn_predictions = []
original_lables_ff_bow = []

with torch.no_grad():
    for index, row in test_df_sub.iterrows():
        bow_vec = make_bow_vector1(review_dict, row['pre_tokens'])
        probs = ff_nn_bow_model(bow_vec)
        bow_ff_nn_predictions.append(torch.argmax(probs, dim=1).cpu().numpy()[0])
        original_lables_ff_bow.append(make_target(true_df['Sentiment'][index]).cpu().numpy()[0])
print(classification_report(original_lables_ff_bow,bow_ff_nn_predictions))
print(' Test accuracy is {}'.format(accuracy_score(original_lables_ff_bow,bow_ff_nn_predictions) * 100))
print(" F1 Score: {:.2f}".format(f1_score(original_lables_ff_bow,bow_ff_nn_predictions, average='macro') * 100))
print(" Precision Score: {:.2f}".format(precision_score(original_lables_ff_bow,bow_ff_nn_predictions, average='macro') * 100))
print(" Recall Score: {:.2f}".format(recall_score(original_lables_ff_bow,bow_ff_nn_predictions, average='macro') * 100))
print("\n")

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       586
           1       0.59      1.00      0.74      1101
           2       0.00      0.00      0.00       172

    accuracy                           0.59      1859
   macro avg       0.20      0.33      0.25      1859
weighted avg       0.35      0.59      0.44      1859

 Test accuracy is 59.225389994620755
 F1 Score: 24.80
 Precision Score: 19.74
 Recall Score: 33.33




/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Task2 using FFNN

In [ ]:
train_df_sub.head()

,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment,processed,tokenized_text,stop_tokens,rem_punct_tokens,pre_tokens
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,1,1,0,0,1,"['look', 'friend', 'lightyear', 'sohalikut', '...","[look, there, my, friend, lightyear, now, all,...","[look, , , friend, lightyear, , , sohalikut, t...","[look, friend, lightyear, sohalikut, trend, pl...","[look, friend, lightyear, sohalikut, trend, pl..."
1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,0,1,0,1,1,"['best', 'yearchallenge', 'complete', 'less', ...","[the, best, of, yearchallenge, completed, in, ...","[, best, , yearchallenge, completed, , , , yea...","[best, yearchallenge, completed, years, kudus,...","[best, yearchalleng, complet, year, kudu, nare..."
2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,1,0,0,0,1,"['sam', 'thorne', 'strippin', 'follow', 'follo...","[sam, thorne, strippin, follow, follow, saw, e...","[sam, thorne, strippin, follow, follow, saw, ,...","[sam, thorne, strippin, follow, follow, saw, p...","[sam, thorn, strippin, follow, follow, saw, po..."
3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,1,1,1,1,1,"['year', 'challenge', 'sweet', 'dee', 'edition']","[year, challenge, sweet, dee, edition]","[year, challenge, sweet, dee, edition]","[year, challenge, sweet, dee, edition]","[year, challeng, sweet, dee, edit]"
4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,1,1,1,0,0,"['year', 'challenge', 'filter', 'hilarious', '...","[year, challenge, with, no, filter, hilarious,...","[year, challenge, , , filter, hilarious, year,...","[year, challenge, filter, hilarious, year, cha...","[year, challeng, filter, hilari, year, challen..."


In [ ]:
# Function to get the output tensor
def make_target_2(label):
    if label == 1:
        return torch.tensor([1], dtype=torch.long, device=device)
    elif label == 0:
        return torch.tensor([0], dtype=torch.long, device=device)

In [ ]:
X_train = train_df_sub[['pre_tokens']]

categories={'humour':0,'sarcasm':1,'offensive':2,'motivational':3}

for category in categories.keys():
  print('******Processing {} category...******'.format(category))

  # Start training
  for epoch in range(num_epochs):
      #if (epoch+1) % 25 == 0:
      #   print("Epoch completed: " + str(epoch+1))
      train_loss = 0
      for index, row in X_train.iterrows():
          # Clearing the accumulated gradients
          optimizer.zero_grad()

          # Make the bag of words vector for stemmed tokens 
          bow_vec = make_bow_vector(review_dict, row['pre_tokens'])
        
          # Forward pass to get output
          probs = ff_nn_bow_model(bow_vec)
          
          # Get the target label
          lab = train_df_sub[category][index]
          target = make_target_2(lab)
          #print(lab)
          #print(target)

          # Calculate Loss: softmax --> cross entropy loss
          loss = loss_function(probs, target)
          # Accumulating the loss over time
          train_loss += loss.item()

          # Getting gradients w.r.t. parameters
          loss.backward()

          # Updating parameters
          optimizer.step()

  bow_ff_nn_predictions = []
  original_lables_ff_bow = []

  with torch.no_grad():
      for index, row in test_df_sub.iterrows():
          bow_vec = make_bow_vector1(review_dict, row['pre_tokens'])
          probs = ff_nn_bow_model(bow_vec)
          bow_ff_nn_predictions.append(torch.argmax(probs, dim=1).cpu().numpy()[0])
          lab = true_df['Labels'].str.split('_').str[1].str[categories[category]][index]
          lab = int(lab)
          #print(lab)
          original_lables_ff_bow.append(make_target_2(lab).cpu().numpy()[0])
  print(classification_report(original_lables_ff_bow,bow_ff_nn_predictions))
  print(' Test accuracy is {}'.format(accuracy_score(original_lables_ff_bow,bow_ff_nn_predictions) * 100))
  print(" F1 Score: {:.2f}".format(f1_score(original_lables_ff_bow,bow_ff_nn_predictions, average='macro') * 100))
  print(" Precision Score: {:.2f}".format(precision_score(original_lables_ff_bow,bow_ff_nn_predictions, average='macro') * 100))
  print(" Recall Score: {:.2f}".format(recall_score(original_lables_ff_bow,bow_ff_nn_predictions, average='macro') * 100))
  print("\n")

******Processing humour category...******


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       443
           1       0.76      1.00      0.86      1416

    accuracy                           0.76      1859
   macro avg       0.38      0.50      0.43      1859
weighted avg       0.58      0.76      0.66      1859

 Test accuracy is 76.16998386229156
 F1 Score: 43.24
 Precision Score: 38.08
 Recall Score: 50.00


******Processing sarcasm category...******
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       418
           1       0.78      1.00      0.87      1441

    accuracy                           0.78      1859
   macro avg       0.39      0.50      0.44      1859
weighted avg       0.60      0.78      0.68      1859

 Test accuracy is 77.51479289940828
 F1 Score: 43.67
 Precision Score: 38.76
 Recall Score: 50.00


******Processing offensive category...******
              precision    recall  f1-score   support

   